<a href="https://colab.research.google.com/github/omiderfanmaneshit/speaker_identification-/blob/omid/ConvertAudioToImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spafe
!pip install --upgrade keras
!pip install "dask[complete]"    # Install everything


In [2]:
import pandas as pd
import numpy as np
import os
from os.path import isfile, join
import matplotlib.pyplot as plt
import librosa.display
import librosa
import scipy
from scipy import signal
from scipy.fftpack import fft
from tqdm import tqdm
from scipy.io import wavfile # get the api
from sklearn import preprocessing
import sklearn 

import pandas as pd
import numpy as np
import os
from os.path import isfile, join
import matplotlib.pyplot as plt
import librosa.display
import librosa
import scipy
from scipy import signal
from scipy.fftpack import fft
from scipy.io import wavfile # get the api
from spafe.utils import vis
from spafe.features.lpc import lpc, lpcc
from spafe.features.mfcc import mfcc, imfcc
from tqdm import tqdm
import dask.dataframe as dd
import dask

# Dependencies
import numpy as np 
import pandas as pd
import os
import librosa
import matplotlib.pyplot as plt
import gc
import time
from tqdm import tqdm, tqdm_notebook; tqdm.pandas() # Progress bar
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.model_selection import train_test_split

# Machine Learning
import tensorflow as tf
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import (Dense, Bidirectional, LSTM, ELU,
                          Dropout, LeakyReLU, Conv1D, BatchNormalization)
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

import csv
import pywt
import cv2 as cvlib
import matplotlib.pyplot as plt
import cv2

In [3]:
def normalize_data_all_gather(vect_in, out_min, out_max, percent_acceptation=80,
                              not_clip_until_acceptation_time_factor=1.5):
    # nb_dim = len(vect_in.shape)
    percent_val = np.percentile(abs(vect_in).reshape((vect_in.shape[0], vect_in.shape[1] * vect_in.shape[2])),
                                percent_acceptation, axis=1)
    percent_val_matrix = not_clip_until_acceptation_time_factor * np.repeat(percent_val,
                                                                            vect_in.shape[1] * vect_in.shape[2],
                                                                            axis=0).reshape(
        (vect_in.shape[0], vect_in.shape[1], vect_in.shape[2]))
    matrix_clip = np.maximum(np.minimum(vect_in, percent_val_matrix), -percent_val_matrix)
    return np.divide(matrix_clip, percent_val_matrix) * ((out_max - out_min) / 2) + (out_max + out_min) / 2

In [4]:
y_label_sound=np.zeros((24449,1))
y_label_hot_sound=np.zeros((24449,10))
n_fft = 1024 # frame length
hop_length = 512

In [8]:
folders_df = []
arr = os.listdir('/content/drive/My Drive/dataScience/september exam/data/development')

for folder in arr:
  if folder == 'j':
    path = '/content/drive/My Drive/dataScience/september exam/data/development/'+folder
    onlyfiles = [f for f in os.listdir(path) if isfile(join(path, f))]
    with tqdm(total=len(onlyfiles), desc="Adding Users", bar_format="{l_bar}{bar} [ time left: {remaining} ]") as pbar:
      for p in onlyfiles:
      
        clipnoise, sample_rate = librosa.load(path +'/'+p,sr=24000)
        scales = np.arange(1, 128)
        waveletname = 'morl'

        coeffnoise, freqnoise = pywt.cwt(clipnoise, scales, waveletname)

        scalogramimg=cvlib.resize(coeffnoise, dsize=(224, 224), interpolation=cvlib.INTER_CUBIC)
        stft = librosa.stft(clipnoise, n_fft=n_fft, hop_length=hop_length)
        stft_magnitude, stft_phase = librosa.magphase(stft)
        stft_magnitude_db = librosa.amplitude_to_db(stft_magnitude, ref=np.max)

        spectrogramimg=cvlib.resize(stft_magnitude_db, dsize=(224, 224), interpolation=cvlib.INTER_CUBIC)

        mfcc = librosa.feature.mfcc(y=clipnoise, sr=sample_rate, n_mfcc=200)
        mfccimg=cvlib.resize(mfcc, dsize=(224, 224), interpolation=cvlib.INTER_CUBIC)

        RGB_sound_VGG = np.zeros(( 224, 224, 3))

        RGB_sound_VGG[ :, :, 0] = spectrogramimg
        RGB_sound_VGG[ :, :, 1] = scalogramimg
        RGB_sound_VGG[ :, :, 2] = mfccimg

        cv2.imwrite('/content/drive/My Drive/dataScience/september exam/data/trainImg' +'/'+folder+'/'+p.split('.')[0]+'.png', RGB_sound_VGG)
        pbar.update(1)

      # RGB_sound_VGG = np.zeros(( 224, 224, 3))
      # RGB_sound_VGG[ :, :, 0] = spectrogramimg
      # RGB_sound_VGG[ :, :, 1] = scalogramimg
      # RGB_sound_VGG[ :, :, 2] = mfccimg

      # scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(0, 255))
      # scaler = scaler.fit(RGB_sound_VGG)
      # X_0255 = scaler.transform(RGB_sound_VGG)

      # cv2.imwrite('/content/drive/My Drive/dataScience/september exam/data/normalizedImg0255' +'/'+folder+'/'+p.split('.')[0]+'.png', X_0255)

      # scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(0, 1))
      # RGB_sound_VGG = np.zeros(( 224, 224, 3))
      # RGB_sound_VGG[ :, :, 0] = spectrogramimg
      # RGB_sound_VGG[ :, :, 1] = scalogramimg
      # RGB_sound_VGG[ :, :, 2] = mfccimg

      
      # scaler = scaler.fit(RGB_sound_VGG)
      # X_01 = scaler.transform(RGB_sound_VGG)

      # cv2.imwrite('/content/drive/My Drive/dataScience/september exam/data/normalizedImg01' +'/'+folder+'/'+p.split('.')[0]+'.png', X_01)


Adding Users: 100%|██████████ [ time left: 00:00 ]
